# Imports

In [2]:
# azure
import yaml
from azure.storage.blob import ContainerClient
from azure.core import exceptions
import re
from azure.servicebus import ServiceBusClient, ServiceBusMessage
from datetime import datetime
import os
import time
from csv import writer


# Functions

## Azure functions

In [3]:

def load_config(dir_config):
    with open(os.path.join(dir_config + r"\config.yaml"), "r") as yamlfile:
        return yaml.load(yamlfile, Loader=yaml.FullLoader)

def upload(files, connection_string, container_name, servicebus_client):
    container_client = ContainerClient.from_connection_string(connection_string, container_name)   
    if not container_client.exists():
        log(servicebus_client,f"creating new containter with {container_name}")
        try:
            container_client.create_container()
        except exceptions.ResourceExistsError:
            log(servicebus_client,"waiting for deletion")
            time.sleep(20)
            container_client.create_container()

    log(servicebus_client,"uploading files:")     
    for file in files:
        blob_client = container_client.get_blob_client(file.name)
        with open(file.path, "rb") as data:
            blob_client.upload_blob(data, overwrite=True)
    log(servicebus_client,f"uploaded {files}")

def send_message(servicebus_client,queue_name, message):
    sender = servicebus_client.get_queue_sender(queue_name=queue_name)
    with sender:  
          #'done--{container_name}--{Bauteil-19.stl}--{True}--{changestring}'
          #changestring=r_n_s_v_slicevariation_mode         else none  
        sender.send_messages(ServiceBusMessage(message))
        log(servicebus_client,f"sent message: {message}")

def get_files(dir):
    with os.scandir(dir) as entries:
        for entry in entries:
            if (entry.is_file()) and ((entry.name.endswith(".stl") or entry.name.endswith(".ply"))):
                yield entry # turns into a generator function

def log(service_bus_client,log,sender_info="LM-"):
    sender = service_bus_client.get_queue_sender(queue_name="log")
    message=sender_info+str(datetime.now())+": "+log
    with sender:  
        sender.send_messages(ServiceBusMessage(message))
        print("log: %s"%(message))

def list_dir_container(config,container_name):
    container_client = ContainerClient.from_connection_string(config["azure_storage_connectionstring"], container_name)
    labels=[]
    if not container_client.exists(): 
        print("wrong connection_string")
        return None
    else:
        blobs=container_client.list_blobs()
        for blob in blobs: labels.append(blob.name)
    return labels

def send_message_to_active_container(servicebus_client,container_list,message):
  for container_num in container_list:
      queue_name_container="queuedpir%s"%(int(container_num))
      sender = servicebus_client.get_queue_sender(queue_name=queue_name_container)
      with sender:  
          sender.send_messages(ServiceBusMessage(message))
          log(servicebus_client,f"sent message: {message}")

def init_containers(servicebus_client, config,container_dict):
    queue_name=config["queue_name"]+"dpir"
    for container_num in list(container_dict.keys()):
        variation=container_dict[container_num][0]
        STL_container=container_dict[container_num][1]
        STL_string="-".join(STL_container)
        init_message="init_container-%s_STL-%s_variation-%s"%(container_num, STL_string,variation)
        send_message(servicebus_client,queue_name, init_message)

def re_init(container_list, container_dict,config, servicebus_client):
    for container_num in container_list:
        queue_name=config["queue_name"]+"dpir"+str(container_num)
        variation=container_dict[container_num][0]
        STL_container=container_dict[container_num][1]
        STL_string="-".join(STL_container)
        init_message="init_container-%s_STL-%s_variation-%s"%(container_num, STL_string,variation)
        send_message(servicebus_client,queue_name, init_message)

def create_container_dict(NUMBER_OF_CONTAINERS,STL_list,VARIATION):
    NUMBER_OF_STL=len(STL_list)
    container_dict={}
    for container_num in range(NUMBER_OF_CONTAINERS):
        STL_per_container=NUMBER_OF_STL/NUMBER_OF_CONTAINERS
        STL_container=STL_list[int(STL_per_container*(container_num)):int(STL_per_container*(container_num+1))]
        container_dict[container_num]=(VARIATION,STL_container)
    return container_dict

def add_containers(NUMBER_OF_CONTAINERS,STL_list,container_dict,config,variation,servicebus_client):
    active_container_list=container_dict.keys()
    container_dict = create_container_dict(NUMBER_OF_CONTAINERS,STL_list)
    container_dict_new={}
    container_dict_old={}
    for keys_new in container_dict_new.keys():
        if not (keys_new in active_container_list):
            container_dict_new[keys_new] = container_dict[keys_new]
        else: 
            container_dict_old[keys_new] = container_dict[keys_new]
    init_containers(servicebus_client, config,container_dict_new)
    re_init(active_container_list,container_dict_old,config,variation,servicebus_client)
    return container_dict


def create_container_group_multi(aci_client, resource_group,
                                 container_group_name,
                                 container_image_1, container_image_2):
    """Creates a container group with two containers in the specified
       resource group.

    Arguments:
        aci_client {azure.mgmt.containerinstance.ContainerInstanceManagementClient}
                    -- An authenticated container instance management client.
        resource_group {azure.mgmt.resource.resources.models.ResourceGroup}
                    -- The resource group in which to create the container group.
        container_group_name {str}
                    -- The name of the container group to create.
        container_image_1 {str}
                    -- The first container image name and tag, for example:
                       microsoft\aci-helloworld:latest
        container_image_2 {str}
                    -- The second container image name and tag, for example:
                       microsoft\aci-tutorial-sidecar:latest
    """
    print("Creating container group '{0}'...".format(container_group_name))

    # Configure the containers
    container_resource_requests = ResourceRequests(memory_in_gb=2, cpu=1.0)
    container_resource_requirements = ResourceRequirements(
        requests=container_resource_requests)

    container_1 = Container(name=container_group_name + '-1',
                            image=container_image_1,
                            resources=container_resource_requirements,
                            ports=[ContainerPort(port=80)])

    container_2 = Container(name=container_group_name + '-2',
                            image=container_image_2,
                            resources=container_resource_requirements)

    # Configure the container group
    ports = [Port(protocol=ContainerGroupNetworkProtocol.tcp, port=80)]
    group_ip_address = IpAddress(
        ports=ports, dns_name_label=container_group_name, type='Public')
    group = ContainerGroup(location=resource_group.location,
                           containers=[container_1, container_2],
                           os_type=OperatingSystemTypes.linux,
                           ip_address=group_ip_address)

    # Create the container group
    aci_client.container_groups.create_or_update(resource_group.name,
                                                 container_group_name, group)

    # Get the created container group
    container_group = aci_client.container_groups.get(resource_group.name,
                                                      container_group_name)

# Execute

In [4]:
config=load_config(".")

## Initialization of Containers

In [5]:
NUMBER_OF_CONTAINERS = 4 #some factor of 24
VARIATION=1
servicebus_client = ServiceBusClient.from_connection_string(conn_str=config['azure_messaging_connectionstring'])
container_name_stl="containerlmim"
labels = list_dir_container(config,container_name_stl)
labels

['Bauteil-006.stl',
 'Bauteil-007.stl',
 'Bauteil-008.stl',
 'Bauteil-009.stl',
 'Bauteil-010.stl',
 'Bauteil-011.stl',
 'Bauteil-012.stl',
 'Bauteil-013.stl',
 'Bauteil-014.stl',
 'Bauteil-015.stl',
 'Bauteil-016.stl',
 'Bauteil-017.stl',
 'Bauteil-018.stl',
 'Bauteil-019.stl',
 'Bauteil-020.stl',
 'Bauteil-022.stl',
 'Bauteil-023.stl',
 'Bauteil-024.stl',
 'Bauteil-025.stl',
 'Bauteil-026.stl',
 'Bauteil-027.stl',
 'Bauteil-028.stl',
 'Bauteil-029.stl',
 'Bauteil-030.stl']

In [6]:
STL_list=[]
for label in labels:
    STL_list.append(str(int(label.split("-")[1].split(".")[0])))
NUMBER_OF_STL=len(STL_list)



In [6]:
 # container num: stl numbers, variation
 

In [21]:
VARIATION=1

In [21]:
container_dict = create_container_dict(NUMBER_OF_CONTAINERS,STL_list,6)
    

In [22]:
container_dict

{0: (6, ['6', '7', '8', '9', '10', '11']),
 1: (6, ['12', '13', '14', '15', '16', '17']),
 2: (6, ['18', '19', '20', '22', '23', '24']),
 3: (6, ['25', '26', '27', '28', '29', '30'])}

In [23]:
def init_containers(servicebus_client, config,container_dict):
    queue_name=config["queue_name"]+"dpir"
    for container_num in list(container_dict.keys())[1:2]:
        variation=container_dict[container_num][0]
        STL_container=container_dict[container_num][1]
        STL_string="-".join(STL_container)
        init_message="init_container-%s_STL-%s_variation-%s"%(container_num, STL_string,variation)
        send_message(servicebus_client,queue_name, init_message)
init_containers(servicebus_client, config,container_dict)
init_containers(servicebus_client, config,container_dict)


log: LM-2022-10-20 04:14:08.716080: sent message: init_container-1_STL-12-13-14-15-16-17_variation-6
log: LM-2022-10-20 04:14:10.741287: sent message: init_container-1_STL-12-13-14-15-16-17_variation-6


In [158]:
# #send message to active containers create queue in azure first

message="id_part48.pcd_containerdpir"
container_list=list(container_dict.keys())[:1]
send_message_to_active_container(servicebus_client,container_list,message)
# # send to id
# active_container_list=container_dict.keys()


log: LM-2022-10-18 01:41:27.903367: sent message: id_part48.pcd_containerdpir


In [62]:
# re_init(active_container_list, container_dict, config)

init_container-0_STL-6-7-8-9-10-11-12-13-14-15-16-17_variation-1
init_container-1_STL-18-19-20-22-23-24-25-26-27-28-29-30_variation-1


In [ ]:
#add new containers
NUMBER_OF_CONTAINERS=4
add_containers(NUMBER_OF_CONTAINERS,STL_list,container_dict)


## Reset Queues

In [111]:

for container_num in list(container_dict)[1:]:
    quit()
    queue_name_container="queuedpir%s"%container_num
    print(queue_name_container)
    it=0
    while it < 5:
        receiver = servicebus_client.get_queue_receiver(queue_name=queue_name_container, max_wait_time=30)
        with receiver:
            messages = receiver.receive_messages(max_wait_time=300)
            for message in messages: 
                receiver.complete_message(message)
                print(message)
        it+=1

queuedpir1
id_part1.pcd_containerdpir
id_part2.pcd_containerdpir
id_part3.pcd_containerdpir
id_part4.pcd_containerdpir
id_part5.pcd_containerdpir
queuedpir2
id_part0.pcd_containerdpir
id_part1.pcd_containerdpir
id_part2.pcd_containerdpir
id_part3.pcd_containerdpir
id_part4.pcd_containerdpir
queuedpir3
id_part0.pcd_containerdpir
id_part1.pcd_containerdpir
id_part2.pcd_containerdpir
id_part3.pcd_containerdpir
id_part4.pcd_containerdpir


## Loop through all Parts and Variations

In [555]:
container_dict = create_container_dict(NUMBER_OF_CONTAINERS,STL_list,11)
init_containers(servicebus_client, config,container_dict)

log: LM-2022-10-18 23:05:31.330945: sent message: init_container-0_STL-6-7-8-9-10-11_variation-11
log: LM-2022-10-18 23:05:33.363999: sent message: init_container-1_STL-12-13-14-15-16-17_variation-11


In [140]:
variaton_number=6
for part_number_load in np.arange(49):
    container_list=list(container_dict.keys())
    container_dict=create_container_dict(NUMBER_OF_CONTAINERS,STL_list,variaton_number)
    re_init(container_list, container_dict,config, servicebus_client)
    message="id_part%s.pcd_containerdpir"%(part_number_load)
    send_message_to_active_container(servicebus_client,container_list,message)

log: LM-2022-10-19 21:59:22.630577: sent message: init_container-0_STL-6-7-8-9-10-11_variation-6
log: LM-2022-10-19 21:59:24.676444: sent message: init_container-1_STL-12-13-14-15-16-17_variation-6
log: LM-2022-10-19 21:59:26.649445: sent message: init_container-2_STL-18-19-20-22-23-24_variation-6
log: LM-2022-10-19 21:59:28.676908: sent message: init_container-3_STL-25-26-27-28-29-30_variation-6
log: LM-2022-10-19 21:59:30.716376: sent message: id_part0.pcd_containerdpir
log: LM-2022-10-19 21:59:32.763631: sent message: id_part0.pcd_containerdpir
log: LM-2022-10-19 21:59:34.790697: sent message: id_part0.pcd_containerdpir
log: LM-2022-10-19 21:59:36.891347: sent message: id_part0.pcd_containerdpir
log: LM-2022-10-19 21:59:38.844060: sent message: init_container-0_STL-6-7-8-9-10-11_variation-6
log: LM-2022-10-19 21:59:40.949697: sent message: init_container-1_STL-12-13-14-15-16-17_variation-6
log: LM-2022-10-19 21:59:42.901974: sent message: init_container-2_STL-18-19-20-22-23-24_varia

In [ ]:
for variaton_number in range(16):
    container_dict=create_container_dict(NUMBER_OF_CONTAINERS,STL_list,variaton_number+1)
    re_init(list(container_dict.keys()), container_dict,config, servicebus_client)
    for part_number_load in range(49):
        message="id_part%s.pcd_containerdpir"%(part_number_load)
        container_list=list(container_dict.keys())
        # send_message_to_active_container(servicebus_client,container_list,message)

## Get results from containers 

### parse results from messages put into csv

In [26]:
from csv import writer
current_dir="."
config=load_config(current_dir)
servicebus_client = ServiceBusClient.from_connection_string(conn_str=config['azure_messaging_connectionstring'])
log_dir=os.path.join(current_dir,"results2","2raw4.csv")
receiver = servicebus_client.get_queue_receiver(queue_name="queueid", max_wait_time=30)
with receiver:
    i=0
    while i<3:
        messages = receiver.receive_messages(max_wait_time=1)
        for message in messages:
            with open(log_dir, 'a+', newline='') as write_obj:
                csv_writer = writer(write_obj)
                csv_writer.writerow([message])
                print(str(message).split("_")[:2])
            receiver.complete_message(message)
            i=0
        i+=1

### Load CSV parse each row load into numpy/ dataframe

In [27]:
import pandas as pd
import numpy as np

In [28]:
def parse_results(raw,factor):
    raw_split=raw.split("_")
    if factor==0: parsed_results=int(raw_split[0].split("-")[1])        #container
    elif factor==1: parsed_results=int(raw_split[1].split("-")[1])      #variation
    elif factor==2: parsed_results=int(raw_split[2].split("-")[1])      #scan
    elif factor==3: parsed_results=float(raw_split[3].split("-")[1])    #time
    elif factor==4: parsed_results=raw_split[4:-1]   
    return parsed_results


stl_num=6
def parse_scores(list_raw,num):
    for part_score in list_raw:
        part_score_split=part_score.split("-")
        if int(part_score_split[1])==int(num):return float(part_score_split[3])
    return None
# STL_list

In [29]:
#load csv

log_dir=os.path.join(".","results2","2raw4.csv")
df=pd.read_csv(log_dir, names=["raw_data"])

#### Parse results

In [30]:
df["container"]=df.apply(lambda row : parse_results(row["raw_data"],0), axis = 1)
df["variation"]=df.apply(lambda row : parse_results(row["raw_data"],1), axis = 1)
df["scan_num"]=df.apply(lambda row : parse_results(row["raw_data"],2), axis = 1)
df["time"]=df.apply(lambda row : parse_results(row["raw_data"],3), axis = 1)
df["raw_score"]=df.apply(lambda row : parse_results(row["raw_data"],4), axis = 1)
for stl_num in STL_list:
    df[stl_num]=df.apply(lambda row : parse_scores(row["raw_score"],int(stl_num)), axis = 1)

temp_csv_dir=os.path.join(".","results2","temp_parsed.csv")
df.to_csv(temp_csv_dir)

In [31]:
number_of_variations=len(pd.unique(df["variation"]))
number_of_scan=len(pd.unique(df["scan_num"]))
number_of_stl=len(STL_list)

#### Calculate Time

In [32]:
#per variation
print("average time: :%s"%(df["time"].mean()))

df_unique_variations=pd.unique(df["variation"])
time_result=np.zeros((len(df_unique_variations)))
for idx_var, variation_unique in enumerate(df_unique_variations):
    time_result[idx_var]=df[df["variation"]==variation_unique]["time"].mean()

time_result_df=pd.DataFrame(data=time_result,columns=["time"], index=df_unique_variations)
time_result_dir=os.path.join(".","results2","time_result.csv")
time_result_df.to_csv(time_result_dir)
time_result_df

average time: :236.23726314549543


,time
5,279.937474
6,192.537052


In [33]:
#per container
df_unique_container=pd.unique(df["container"])
time_result_cont=np.zeros((len(df_unique_container)))
for idx_cont, container_unique in enumerate(df_unique_container):
    time_result_cont[idx_cont]=df[df["container"]==container_unique]["time"].mean()

pd.DataFrame(data=time_result_cont,columns=["time"], index=df_unique_container).sort_values(["time"])


,time
0,208.427001
2,219.083180
3,236.221585
1,281.217287


#### COMBINE ROWS

In [35]:
# new_data=np.zeros((number_of_variations,number_of_scan,number_of_stl))
number_of_variations=16
result=np.zeros((number_of_variations,number_of_scan))
missing_cont=np.ones((number_of_variations,number_of_scan,4))*-1
missing=np.zeros((number_of_variations,number_of_scan))
df_unique_comb=df.groupby(["variation", "scan_num"]).size().reset_index()
for idx, comb in df_unique_comb.iterrows():
    
    score_df =df[(df["variation"]==comb["variation"]) & (df["scan_num"]==comb["scan_num"])].get(STL_list)
    #remove double
    data_comb=np.zeros((4,24))
    unique_container_list=[]
    for container_row_idx, container_row in df[(df["variation"]==comb["variation"]) & (df["scan_num"]==comb["scan_num"])].iterrows():
        if not (container_row["container"] in unique_container_list):
            unique_container_list.append(container_row["container"])
            data_comb[container_row["container"],:]=container_row.get(STL_list).to_numpy(np.dtype("float"))
    score_df=np.nansum(data_comb,axis=0)
    if comb[0]>=4:
        result[comb["variation"]-1,comb["scan_num"]]=int(STL_list[np.argmin(score_df)])
        missing_cont[comb["variation"]-1,comb["scan_num"],:]=np.arange(4)
    else:
        missing_cont[comb["variation"]-1,comb["scan_num"],np.unique(df[(df["variation"]==comb["variation"]) & (df["scan_num"]==comb["scan_num"])]["container"])]=np.unique(df[(df["variation"]==comb["variation"]) & (df["scan_num"]==comb["scan_num"])]["container"])
        missing[comb["variation"]-1,comb["scan_num"]]=1
   # new_data[comb["variation"]-1,comb["scan_num"],:]=score_df

results_df=pd.DataFrame(data=result,index=np.arange(start=1,stop=number_of_variations+1),columns=np.arange(stop=number_of_scan))


missing_df=pd.DataFrame(data=missing,index=np.arange(start=1,stop=number_of_variations+1),columns=np.arange(stop=number_of_scan))
missing_csv_dir=os.path.join(".","results2","missing.csv")
missing_df.to_csv(missing_csv_dir)

temp_csv_dir=os.path.join(".","results2","results.csv")
results_df.to_csv(temp_csv_dir)

# results_df.index.name="variation"

#### LOAD CSV of scan to stl num

In [36]:
df_scan_stl =pd.read_csv("scans.csv",header=0)
stl_nums=np.sort(df_scan_stl["STL"].unique())

#### Calculate accuracy

In [37]:
def calc_accuracy(row,df_scan_stl):
    non_zero_index=np.arange(len(row))[row!=0]
    non_zero_results=row[non_zero_index].to_numpy(np.dtype(np.int16))
    non_zero_target=df_scan_stl.sort_values("Part")["STL"].to_numpy(np.dtype(np.int16))[non_zero_index]
    difference=non_zero_results-non_zero_target
    if len(difference)!=0:
        accuracy=np.ones((len(difference)))[difference==0].sum()/(len(difference))
        return accuracy
    else:
        return None

In [38]:
results_df["accuracy"]=results_df.apply(lambda row : calc_accuracy(row,df_scan_stl), axis = 1)
results_df.to_csv(temp_csv_dir)
results_df["accuracy"]

1          NaN
2          NaN
3          NaN
4          NaN
5     0.183673
6     0.183673
7          NaN
8          NaN
9          NaN
10         NaN
11         NaN
12         NaN
13         NaN
14         NaN
15         NaN
16         NaN
Name: accuracy, dtype: float64

In [39]:
results_df

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,accuracy
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
5,10.0,25.0,20.0,20.0,19.0,20.0,19.0,20.0,29.0,19.0,...,23.0,14.0,25.0,26.0,14.0,12.0,29.0,25.0,14.0,0.183673
6,10.0,29.0,20.0,9.0,19.0,25.0,20.0,20.0,9.0,28.0,...,12.0,14.0,14.0,12.0,9.0,20.0,20.0,14.0,29.0,0.183673
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [40]:
missing_df

,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
